In [147]:
from bloqade import qasm2
from bloqade.qasm2.emit import QASM2 # the QASM2 target
import numpy as np
import math
import itertools
import warnings
warnings.filterwarnings("ignore")

from pyqrack import QrackSimulator
from bloqade.pyqrack import PyQrack, reg

from bloqade.qasm2.parse import pprint # the QASM2 pretty printer
import sys
sys.path.append('/Users/harrywanghc/Developer/2025/2025YaleQHack/src/')
from PauliHamiltonian import (
    PauliOp, PauliTerm, PauliHamiltonian
)

from collections import Counter
from bloqade.qasm2.rewrite.native_gates import RydbergGateSetRewriteRule
from kirin import ir
from kirin.rewrite import Walk
from bloqade.qasm2.passes import UOpToParallel, QASM2Fold

In [148]:
def create_schwinger_hamiltonian(N: int, m: float = 1.0, g: float = 1.0, a: float = 1.0) -> PauliHamiltonian:
    """
    Create the Schwinger Hamiltonian for 1+1D quantum electrodynamics.
    
    Args:
        N: Number of lattice sites
        m: Mass parameter
        g: Coupling constant
        a: Lattice spacing
    
    Returns:
        PauliHamiltonian representing the Schwinger model
    """
    ham = PauliHamiltonian()
    
    # Hopping terms (XX and YY interactions)
    for n in range(1, N):
        # XX terms: (1/4a) * X_{n-1} X_n
        ham.add_term(PauliTerm(1/(4*a), {n-1: PauliOp.X, n: PauliOp.X}))
        
        # YY terms: (1/4a) * Y_{n-1} Y_n
        ham.add_term(PauliTerm(1/(4*a), {n-1: PauliOp.Y, n: PauliOp.Y}))
    
    # Mass terms: (-1)^n * (m/2) * Z_{n-1}
    for n in range(1, N+1):
        coefficient = ((-1)**n) * (m/2)
        ham.add_term(PauliTerm(coefficient, {n-1: PauliOp.Z}))
    
    # Electric field energy terms
    c_ga = (a * g**2 / 2) * (1/4)  # Common coefficient
    
    for n in range(1, N):
        # First part: Identity terms
        for j in range(1, n+1):
            # Identity term: c_ga * 2 * I_{j-1}
            ham.add_term(PauliTerm(c_ga * 2, {}))  # Empty dict means identity on all qubits
            
            # Z term: c_ga * 2 * (-1)^j * Z_{j-1}
            coefficient = c_ga * 2 * ((-1)**j)
            ham.add_term(PauliTerm(coefficient, {j-1: PauliOp.Z}))
        
        # Second part: ZZ and mixed terms
        for l in range(1, n+1):
            for m_idx in range(1, l):
                # ZZ term: c_ga * 2 * Z_{l-1} Z_{m-1}
                ham.add_term(PauliTerm(c_ga * 2, {l-1: PauliOp.Z, m_idx-1: PauliOp.Z}))
                
                # Identity term: c_ga * 2 * (-1)^(l+m) * I
                coefficient = c_ga * 2 * ((-1)**(l+m_idx))
                ham.add_term(PauliTerm(coefficient, {}))
                
                # Z term on m: c_ga * 2 * (-1)^l * Z_{m-1}
                coefficient = c_ga * 2 * ((-1)**l)
                ham.add_term(PauliTerm(coefficient, {m_idx-1: PauliOp.Z}))
                
                # Z term on l: c_ga * 2 * (-1)^m * Z_{l-1}
                coefficient = c_ga * 2 * ((-1)**m_idx)
                ham.add_term(PauliTerm(coefficient, {l-1: PauliOp.Z}))
    
    return ham.simplify()

In [149]:
H_test = create_schwinger_hamiltonian(4, 1, 1, 1)
print(H_test)
H_matrix = H_test.to_matrix()
eigs = np.linalg.eigvalsh(H_matrix)
print("Eigenvalues:", min(eigs))


0.25*X_0 X_1 + 0.25*Y_0 Y_1 + 0.25*X_1 X_2 + 0.25*Y_1 Y_2 + 0.25*X_2 X_3 + 0.25*Y_2 Y_3 + -1.0*Z_0 + 0.25*Z_1 + -0.75*Z_2 + 0.5*Z_3 + 1.0*I + 0.5*Z_0 Z_1 + 0.25*Z_0 Z_2 + 0.25*Z_1 Z_2
Eigenvalues: -2.2765645864303945


In [150]:
@ir.dialect_group(qasm2.extended)
def extended_opt(self):
    native_rewrite = Walk(RydbergGateSetRewriteRule(self)) # use Kirin's functionality to walk code line by line while applying neutral-atom gate decomposition as defined in Bloqade
    parallelize_pass = UOpToParallel(self) # review the code and apply parallelization using a heuristic
    agg_fold = QASM2Fold(self) # supports parallelization by unfolding loops to search for parallelization opportunities

    # here we define our new compiler pass
    def run_pass(
        kernel: ir.Method,
        *,
        fold: bool = True,
        typeinfer: bool = True,
        parallelize: bool = False,
    ):
        assert qasm2.extended.run_pass is not None
        qasm2.extended.run_pass(kernel, fold=fold, typeinfer=typeinfer) # apply the original run_pass to the lowered kernel
        native_rewrite.rewrite(kernel.code) # decompose all gates in the circuit to neutral atom gate set

        # here goes our parallelization optimizer; the order of the commands here matters!
        if parallelize:
            agg_fold.fixpoint(kernel)
            parallelize_pass(kernel)

    return run_pass

In [151]:
def ansatz_no_ham(params, n_qubits: int, reps: int, skip_final_rotation_layer=False):
    # entangle_pairs = list(itertools.combinations(range(n_qubits), 2))
    
    @extended_opt
    def ansatz():
        
        qreg = qasm2.qreg(n_qubits)
        creg = qasm2.creg(n_qubits)
        for r in range(reps):
            # Apply the rotation blocks
            for i in range(n_qubits):
                qasm2.rx(qreg[i], params[r, i, 0])
                qasm2.ry(qreg[i], params[r, i, 1])
                qasm2.rz(qreg[i], params[r, i, 2])
            if r < reps:
                for j in range(n_qubits):
                    for k in range(j+1, n_qubits):
                        qasm2.cx(qreg[j], qreg[k])
                        qasm2.cz(qreg[j], qreg[k])
        # Apply the final rotation layer
        if not skip_final_rotation_layer:
            for i in range(n_qubits):
                qasm2.rx(qreg[i], params[-1, i, 0])
                qasm2.ry(qreg[i], params[-1, i, 1])
                qasm2.rz(qreg[i], params[-1, i, 2])
        # Hamiltonian layer: No Hamiltonian layer in this ansatz
        for i in range(n_qubits):
            qasm2.measure(qreg[i],creg[i])

        return creg
    
    return ansatz

In [152]:
kernel = ansatz_no_ham(init_params, num_qubits, layers)

In [153]:
device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
results = device.multi_run(kernel, _shots=100)

def to_bitstrings(results):
        return Counter(map(lambda result:"".join(map(str, result)), results))
    
counts = to_bitstrings(results)
counts



Counter({'1011': 22,
         '0110': 15,
         '0111': 11,
         '1010': 7,
         '0101': 6,
         '0011': 6,
         '0100': 5,
         '1000': 5,
         '0001': 5,
         '1111': 4,
         '1100': 3,
         '1101': 3,
         '1001': 3,
         '0000': 3,
         '0010': 2})

In [154]:
def counter_measure(counts, operators=None):
    """
    Calculate the expectation value of a multi-qubit Z operator from measurement counts.
    
    Args:
        counts: Counter object with bitstrings as keys and counts as values
        operators: List of operator indices to consider (defaults to all qubits)
                    For example, [0, 2, 3] would measure Z⊗I⊗Z⊗Z
    
    Returns:
        The expectation value of the Z operator product
    """
    total_counts = sum(counts.values())
    
    # Determine the number of qubits from the first bitstring
    n_qubits = len(next(iter(counts.keys())))
    
    # If no specific operators provided, use all qubits
    if operators is None:
        operators = list(range(n_qubits))
    
    # Calculate expectation value
    expectation = 0.0
    for bitstring, count in counts.items():
        # Convert each bit to ±1 (0→+1, 1→-1) and multiply
        # only for the specified qubits
        z_product = 1
        for idx in operators:
            if idx < len(bitstring):  # Ensure the index is valid
                z_product *= (-1)**int(bitstring[idx])
        
        # Add to expectation, weighted by count
        expectation += z_product * count
    
    # Normalize by total counts
    return expectation / total_counts

# Measure ZZZZ (all qubits)
zzzz_expectation = counter_measure(counts)
print(f"ZZZZ expectation value: {zzzz_expectation}")

ZZZZ expectation value: -0.06


In [155]:
for terms in H_test.terms:
    print(terms.coefficient)
    print(terms.operators)

0.25
{0: X, 1: X}
0.25
{0: Y, 1: Y}
0.25
{1: X, 2: X}
0.25
{1: Y, 2: Y}
0.25
{2: X, 3: X}
0.25
{2: Y, 3: Y}
-1.0
{0: Z}
0.25
{1: Z}
-0.75
{2: Z}
0.5
{3: Z}
1.0
{}
0.5
{0: Z, 1: Z}
0.25
{0: Z, 2: Z}
0.25
{1: Z, 2: Z}


In [166]:
def ansatz_with_ham_exp(params, hamiltonian, n_qubits: int, reps: int, skip_final_rotation_layer=False, shots=100):
    # entangle_pairs = list(itertools.combinations(range(n_qubits), 2))
    reshaped_params = params.reshape(reps+1, n_qubits, 3)
    params = reshaped_params
    # for each subterm in hamiltonian, we apply the corresponding gates.
    # That is, if we have a term.operator like X_0 Y_1, we apply the corresponding gates: 
    # X_0: add hadamard to qubit 0 to make it Z.
    # Y_1: add Sdg to qubit 1 and then H to make it Z.
    # Z_2: No change to qubit 2.
    # I_3: No change to qubit 3. If it's empty, then it's Identity. We do nothing
    # The Hamiltonian is a sum of terms, so we can apply each term in parallel.

    expectation_value = 0

    coeff_list = [terms.coefficient for terms in hamiltonian.terms]
    
    # 0/13
    def sub_ansatz0():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {0: X, 1: X}
            qasm2.h(qreg[0])
            qasm2.h(qreg[1])
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    
    sub_kernel0 = sub_ansatz0()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results0 = device.multi_run(sub_kernel0, _shots=shots)
    counts0 = to_bitstrings(results0)
    expectation0 = counter_measure(counts0, operators=[0, 1])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[0] * expectation0 

    # 1/13
    def sub_ansatz1():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {0: Y, 1: Y}
            qasm2.sdg(qreg[0])
            qasm2.h(qreg[0])
            qasm2.sdg(qreg[1])
            qasm2.h(qreg[1])
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    
    sub_kernel1 = sub_ansatz1()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results1 = device.multi_run(sub_kernel1, _shots=shots)
    counts1 = to_bitstrings(results1)
    expectation1 = counter_measure(counts1, operators=[0, 1])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[1] * expectation1

    # 2/13
    def sub_ansatz2():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {1: X, 2: X}
            qasm2.h(qreg[1])
            qasm2.h(qreg[2])
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    sub_kernel2 = sub_ansatz2()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results2 = device.multi_run(sub_kernel2, _shots=shots)
    counts2 = to_bitstrings(results2)
    expectation2 = counter_measure(counts2, operators=[1, 2])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[2] * expectation2
    
    # 3/13 {1: Y, 2: Y}
    def sub_ansatz3():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {1: Y, 2: Y}
            qasm2.sdg(qreg[1])
            qasm2.h(qreg[1])
            qasm2.sdg(qreg[2])
            qasm2.h(qreg[2])
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    sub_kernel3 = sub_ansatz3()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results3 = device.multi_run(sub_kernel3, _shots=shots)
    counts3 = to_bitstrings(results3)
    expectation3 = counter_measure(counts3, operators=[1, 2])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[3] * expectation3

    # 4/13 {2: X, 3: X}
    def sub_ansatz4():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {2: X, 3: X}
            qasm2.h(qreg[2])
            qasm2.h(qreg[3])
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    sub_kernel4 = sub_ansatz4()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results4 = device.multi_run(sub_kernel4, _shots=shots)
    counts4 = to_bitstrings(results4)
    expectation4 = counter_measure(counts4, operators=[2, 3])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[4] * expectation4

    # 5/13 {2: Y, 3: Y}
    def sub_ansatz5():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {2: Y, 3: Y}
            qasm2.sdg(qreg[2])
            qasm2.h(qreg[2])
            qasm2.sdg(qreg[3])
            qasm2.h(qreg[3])
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    sub_kernel5 = sub_ansatz5()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results5 = device.multi_run(sub_kernel5, _shots=shots)
    counts5 = to_bitstrings(results5)
    expectation5 = counter_measure(counts5, operators=[2, 3])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[5] * expectation5

    # 6/13 {0: Z}
    def sub_ansatz6():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {0: Z}
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    sub_kernel6 = sub_ansatz6()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results6 = device.multi_run(sub_kernel6, _shots=shots)
    counts6 = to_bitstrings(results6)
    expectation6 = counter_measure(counts6, operators=[0])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[6] * expectation6

    # 7/13 {1: Z}
    def sub_ansatz7():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {1: Z}
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    sub_kernel7 = sub_ansatz7()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results7 = device.multi_run(sub_kernel7, _shots=shots)
    counts7 = to_bitstrings(results7)
    expectation7 = counter_measure(counts7, operators=[1])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[7] * expectation7

    # 8/13 {2: Z}
    def sub_ansatz8():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {2: Z}
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    sub_kernel8 = sub_ansatz8()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results8 = device.multi_run(sub_kernel8, _shots=shots)
    counts8 = to_bitstrings(results8)
    expectation8 = counter_measure(counts8, operators=[2])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[8] * expectation8

    # 9/13 {3: Z}   
    def sub_ansatz9():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {3: Z}
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    sub_kernel9 = sub_ansatz9()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results9 = device.multi_run(sub_kernel9, _shots=shots)
    counts9 = to_bitstrings(results9)
    expectation9 = counter_measure(counts9, operators=[3])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[9] * expectation9

    # 10/13 {}
    expectation_value += coeff_list[10] * 1

    # 11/13 {0: Z, 1: Z}
    def sub_ansatz11():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {0: Z, 1: Z}
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    sub_kernel11 = sub_ansatz11()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results11 = device.multi_run(sub_kernel11, _shots=shots)
    counts11 = to_bitstrings(results11)
    expectation11 = counter_measure(counts11, operators=[0, 1])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[11] * expectation11

    # 12/13 {0: Z, 2: Z}
    def sub_ansatz12():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {0: Z, 2: Z}
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    sub_kernel12 = sub_ansatz12()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results12 = device.multi_run(sub_kernel12, _shots=shots)
    counts12 = to_bitstrings(results12)
    expectation12 = counter_measure(counts12, operators=[0, 2])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[12] * expectation12

    # 13/13 {1: Z, 2: Z}
    def sub_ansatz13():

        @extended_opt
        def ansatz():
            
            qreg = qasm2.qreg(n_qubits)
            creg = qasm2.creg(n_qubits)
            for r in range(reps):
                # Apply the rotation blocks
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[r, i, 0])
                    qasm2.ry(qreg[i], params[r, i, 1])
                    qasm2.rz(qreg[i], params[r, i, 2])
                if r < reps:
                    for j in range(n_qubits):
                        for k in range(j+1, n_qubits):
                            qasm2.cx(qreg[j], qreg[k])
                            qasm2.cz(qreg[j], qreg[k])
            # Apply the final rotation layer
            if not skip_final_rotation_layer:
                for i in range(n_qubits):
                    qasm2.rx(qreg[i], params[-1, i, 0])
                    qasm2.ry(qreg[i], params[-1, i, 1])
                    qasm2.rz(qreg[i], params[-1, i, 2])
            # Hamiltonian layer: {1: Z, 2: Z}
            # Measurement
            for i in range(n_qubits):
                qasm2.measure(qreg[i],creg[i])

            return creg
        
        return ansatz
    sub_kernel13 = sub_ansatz13()
    device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
    results13 = device.multi_run(sub_kernel13, _shots=shots)
    counts13 = to_bitstrings(results13)
    expectation13 = counter_measure(counts13, operators=[1, 2])
    # Multiply by the coefficient of the term
    expectation_value += coeff_list[13] * expectation13

    return expectation_value

In [157]:
# test this with params.
num_qubits=4
layers=1
init_params = np.random.random((layers+1, num_qubits, 3))*2*np.pi

In [158]:
e_val = ansatz_with_ham_exp(H_test, init_params, num_qubits, layers, shots=300)

In [159]:
print(e_val)

1.2699999999999998


In [160]:
from scipy.optimize import minimize

In [170]:
def vqe(hamiltonian, shots=300, maxiter = 300, random_seed = 42, verbose = False):
    # helper func
    exp_val_list = []
    # def callback(num_eval, params, f_eval, step, acc):
    def callback(params):
        exp_val = ansatz_with_ham_exp(params, hamiltonian, num_qubits, layers, shots=shots)
        exp_val_list.append(exp_val)
        if verbose == True: print(f"step {len(exp_val_list):>4}:  <H> = {exp_val:>6.3f}")

    def minimize_free_energy(params):
        value_to_minimize = ansatz_with_ham_exp(params, hamiltonian, num_qubits, layers, shots=shots)
        return value_to_minimize
    
    np.random.seed(random_seed)
    inital_params = np.random.random((layers+1, num_qubits, 3))*2*np.pi
    flatten_initial_params = inital_params.flatten()
    opt = minimize(minimize_free_energy, flatten_initial_params, method='COBYLA', callback=callback, options={'maxiter': maxiter})

    result = opt.minimize(minimize_free_energy, inital_params)

    return result, exp_val_list

In [162]:
vqe_result, exp_val_list = vqe(H_test, shots=200, maxiter=300, random_seed=42, verbose=True)

step    1:  <H> =  1.300
step    2:  <H> =  1.283
step    3:  <H> =  0.983
step    4:  <H> =  1.583
step    5:  <H> =  1.100
step    6:  <H> =  1.567
step    7:  <H> =  0.567
step    8:  <H> =  0.967
step    9:  <H> =  0.850
step   10:  <H> =  0.450
step   11:  <H> =  1.483
step   12:  <H> =  1.017
step   13:  <H> =  2.000
step   14:  <H> =  0.467
step   15:  <H> =  1.500
step   16:  <H> =  0.850
step   17:  <H> =  0.950
step   18:  <H> =  0.717
step   19:  <H> =  1.017
step   20:  <H> =  0.633
step   21:  <H> =  1.350
step   22:  <H> =  0.417
step   23:  <H> =  1.167
step   24:  <H> =  1.100
step   25:  <H> =  1.167
step   26:  <H> =  0.250
step   27:  <H> =  1.700
step   28:  <H> =  0.383
step   29:  <H> =  0.083
step   30:  <H> =  1.700
step   31:  <H> =  0.800
step   32:  <H> =  0.950
step   33:  <H> =  1.933
step   34:  <H> =  1.183
step   35:  <H> =  0.900
step   36:  <H> =  1.100
step   37:  <H> =  1.000
step   38:  <H> =  0.800
step   39:  <H> =  1.483
step   40:  <H> =  1.217


KeyboardInterrupt: 

In [ ]:
opt_params = vqe_result.x.reshape(layers+1, num_qubits, 3)
opt_energy = vqe_result.fun

In [ ]:
print(exp_val_list)

In [ ]:
vqe_result, exp_val_list = vqe(H_test, shots=200, maxiter=300, random_seed=42, verbose=True)

step    1:  <H> =  1.100
step    2:  <H> =  1.353
step    3:  <H> =  1.020
step    4:  <H> =  0.855
step    5:  <H> =  0.765
step    6:  <H> =  0.745
step    7:  <H> = -0.167
step    8:  <H> = -0.290
step    9:  <H> =  0.735
step   10:  <H> =  0.188
step   11:  <H> = -0.112
step   12:  <H> = -0.088
step   13:  <H> = -0.077
step   14:  <H> = -0.800
step   15:  <H> = -0.695
step   16:  <H> = -0.605
step   17:  <H> = -0.760
step   18:  <H> = -0.540
step   19:  <H> = -0.682
step   20:  <H> = -0.140
step   21:  <H> = -0.763
step   22:  <H> = -0.828
step   23:  <H> = -0.955
step   24:  <H> = -0.885
step   25:  <H> = -0.897
step   26:  <H> = -0.982
step   27:  <H> = -0.400
step   28:  <H> = -1.180
step   29:  <H> = -0.575
step   30:  <H> = -0.900
step   31:  <H> = -0.855
step   32:  <H> = -0.733
step   33:  <H> = -0.950
step   34:  <H> = -0.755
step   35:  <H> = -1.073
step   36:  <H> = -1.202
step   37:  <H> = -1.250
step   38:  <H> = -1.502
step   39:  <H> = -1.507
step   40:  <H> = -1.235
